<a href="https://colab.research.google.com/github/Multi-Objective-NAS/invertible-nas-algorithm/blob/master/Neural_ODE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installation

In [0]:
pip install git+https://github.com/rtqichen/torchdiffeq

  Cloning https://github.com/rtqichen/torchdiffeq to /tmp/pip-req-build-m3tothpt
  Running command git clone -q https://github.com/rtqichen/torchdiffeq /tmp/pip-req-build-m3tothpt
  Created wheel for torchdiffeq: filename=torchdiffeq-0.0.1-cp36-none-any.whl size=27578 sha256=11d6e7724a36aa22658c2cc634f8d34d077ef1ecea710dc31ba5effb22d82d8a
  Stored in directory: /tmp/pip-ephem-wheel-cache-u00txpge/wheels/3f/76/69/01867bf3355c3bc8bae7e556b17b44c395b6cda5e76fd8ddc7
Successfully built torchdiffeq


In [0]:
!git clone https://github.com/rtqichen/torchdiffeq

Cloning into 'torchdiffeq'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 225 (delta 2), reused 2 (delta 0), pack-reused 214
Receiving objects: 100% (225/225), 712.88 KiB | 15.50 MiB/s, done.
Resolving deltas: 100% (99/99), done.


In [0]:
cd torchdiffeq/_impl

[Errno 2] No such file or directory: 'torchdiffeq/_impl'
/content


#Installation dataset

In [0]:
#!curl -O https://storage.googleapis.com/nasbench/nasbench_full.tfrecord
!curl -O https://storage.googleapis.com/nasbench/nasbench_only108.tfrecord
# Clone and install the code and dependencies.

!git clone https://github.com/google-research/nasbench
!pip install ./nasbench

# Initialize the NASBench object which parses the raw data into memory (this
# should only be run once as it takes up to a few minutes).

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  498M  100  498M    0     0  78.0M      0  0:00:06  0:00:06 --:--:-- 91.5M
Cloning into 'nasbench'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 96 (delta 0), reused 2 (delta 0), pack-reused 92
Unpacking objects: 100% (96/96), done.
Processing ./nasbench
  Created wheel for nasbench: filename=nasbench-1.0-cp36-none-any.whl size=46787 sha256=63873caefe040242d3a5aa9da8794caf7a9be09312985515fcb9a73a4ac4b534
  Stored in directory: /tmp/pip-ephem-wheel-cache-85hhwt1m/wheels/4b/19/99/1d5fdfe30f8b16fab91e900808f4f7e5adc38e602c84970ad5
Successfully built nasbench


#Setting

In [0]:
!pwd

/content


In [0]:
import os
import argparse
import logging
import time
import numpy as np
import torch
import torch.nn as nn
import random
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [0]:
from torchdiffeq import odeint

In [0]:
# main argument
#Device = torch.device('cuda:0')
Device = torch.device('cpu')
Is_odenet = True
downsampling_method = 'conv'
Batch_size = 50
Lr = 0.1
NEpochs = 1
Rtol=1e-7
Atol=1e-9
Edge_num = 9

In [0]:
from nasbench import api
#nasbench = api.NASBench('nasbench_full.tfrecord')
nasbench = api.NASBench('nasbench_only108.tfrecord')




Loading dataset from file... This may take a few minutes...
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Loaded dataset in 57 seconds


In [0]:
# Useful constants
INPUT = 'input'
OUTPUT = 'output'
CONV3X3 = 'conv3x3-bn-relu'
CONV1X1 = 'conv1x1-bn-relu'
MAXPOOL3X3 = 'maxpool3x3'
OUTPUT = 'output'
NUM_VERTICES = 7
MAX_EDGES = 9
EDGE_SPOTS = NUM_VERTICES * (NUM_VERTICES - 1) / 2  # Upper triangular matrix
OP_SPOTS = NUM_VERTICES - 2  # Input/output vertices are fixed
ALLOWED_OPS = [CONV3X3, CONV1X1, MAXPOOL3X3, OUTPUT]
ALLOWED_EDGES = [0, 1]  # Binary adjacency matrix

In [0]:
Embed_size = 7 + 7 + len(ALLOWED_OPS)

#Code

In [0]:
def encode(mat, ops):
    def _sanity_check(idx1, opidx, idx2):
        if idx1 not in range(7):
            return False
        if idx2 not in range(7):
            return False
        if opidx not in range(4):
            return False
        return True

    encoded = []
    for inbound, outbound in zip(*np.array(mat).nonzero()):
        op = ALLOWED_OPS.index(ops[outbound])
        assert _sanity_check(inbound, op, outbound)

        embed = [0] * (EMBED_SIZE)
        embed[inbound] = 1
        embed[7 + op] = 1
        embed[7 + len(ALLOWED_OPS) + outbound] = 1

        encoded.append(embed)

    assert np.array(encoded).shape == (9, EMBED_SIZE)
    return encoded

def max_index(output):
    def find_max_index(ls):
        return ls.index(max(ls))

    encoded = []
    outputs = [output[i: i + EMBED_SIZE] for i in range(0,len(output), EMBED_SIZE)]
    for output in outputs:
        embed = [0 for _ in range(EMBED_SIZE)]
        inbound, op, outbound = find_max_index(output[:7]), find_max_index(output[7:-7]), find_max_index(output[-7:])
        embed[inbound] = 1
        embed[7 + op] = 1
        embed[7 + len(ALLOWED_OPS) + outbound] = 1
        encoded += embed
    
    assert np.sum(np.array(encoded)) == 3
    return encoded

def decode(encoded):
    mat = np.zeros((7,7))
    op = [INPUT] + [CONV3X3 for _ in range(5)] + [OUTPUT]

    outputs = [output[i: i + EMBED_SIZE] for i in range(0,len(output), EMBED_SIZE)]
    for embed in outputs:
        inbound, op, outbound = np.nonzero(np.array(embed))[0]
        op -= 7
        outbound -= (7+len(ALLOWED_OPS))
        mat[inbound][outbound] = 1
        op[outboung] = ALLOWED_OPS[op]
    
    return mat.tolist(), op

In [0]:
assert np.array([[1,2,3],[1,2,3]]).shape == (2,3)

In [0]:
#data_loader: x = embedding code, y = (accuracy, latency)
#return train_loader, test_loader, train_eval_loader
def generate_data(batch_size=100, validation_ratio=0.1, test_ratio=0.1):
    def run_experiment(mat, op):
        model_spec = api.ModelSpec(matrix=mat, ops=op)
        q = nasbench.query(model_spec)

        return (q['validation_accuracy'], q['training_time'])
    
    def train_val_test_split(dataset):
        random.shuffle(dataset)
        total_size = (len(dataset) // batch_size) * batch_size
        dataset = dataset[: total_size]
        val_size = int(total_size * validation_ratio)
        test_size = int(total_size * test_ratio)

        val, dataset = dataset[:val_size], dataset[val_size:]
        test, train = dataset[:test_size], dataset[test_size:]

        return train, val, test
    
    def is_target_graph(mat, op):
        return np.sum(mat) == Edge_num


    # Find every possible graph
    dataset = []
    for hash_val in nasbench.hash_iterator():
        fixed_stat, _ = nasbench.get_metrics_from_hash(hash_val)
        mat, op = fixed_stat['module_adjacency'], fixed_stat['module_operations']
        if is_target_graph(mat, op):
            x = encode(mat, op)
            y = run_experiment(mat, op)
            dataset.append((x,y))

    return train_val_test_split(dataset)

def data_loader(dataset, batch_turn, size):
    xset = []
    yset = []
    for x,y in dataset[size * batch_turn : size * batch_turn + size]:
        xset.append(x)
        yset.append(y)
    xset = torch.FloatTensor(xset)
    yset = torch.FloatTensor(yset)
    return xset, yset

In [0]:
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


def norm(dim):
    return nn.GroupNorm(min(32, dim), dim)

In [0]:
class ConcatConv1d(nn.Module):

    def __init__(self, dim_in, dim_out, ksize=3, stride=1, padding=0, dilation=1, groups=1, bias=True):
        super(ConcatConv1d, self).__init__()
        self._layer = nn.Conv1d(
            dim_in + 1, dim_out, kernel_size=ksize, stride=stride, padding=padding, dilation=dilation, groups=groups,
            bias=bias
        )

    def forward(self, t, x):
        tt = torch.ones_like(x[:, :1, :]) * t
        ttx = torch.cat([tt, x], 1)
        return self._layer(ttx)


class ODEfunc(nn.Module):

    def __init__(self, dim):
        super(ODEfunc, self).__init__()
        self.norm1 = norm(dim)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = ConcatConv1d(dim, dim, 3, 1, 1)
        self.norm2 = norm(dim)
        self.conv2 = ConcatConv1d(dim, dim, 3, 1, 1)
        self.norm3 = norm(dim)
        self.nfe = 0

    def forward(self, t, x):
        self.nfe += 1
        out = self.norm1(x)
        out = self.relu(out)
        out = self.conv1(t, out)
        out = self.norm2(out)
        out = self.relu(out)
        out = self.conv2(t, out)
        out = self.norm3(out)
        return out

In [0]:
'''
class ODEfunc(nn.Module):

    def __init__(self, dim):
        super(ODEfunc, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(dim, 50),
            nn.Tanh(),
            nn.Linear(50, dim),
        )
        self.nfe = 0

        for m in self.net.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, mean=0, std=0.1)
                nn.init.constant_(m.bias, val=0)

    def forward(self, t, x):
        self.nfe += 1
        return self.net(x)
'''

'\nclass ODEfunc(nn.Module):\n\n    def __init__(self, dim):\n        super(ODEfunc, self).__init__()\n\n        self.net = nn.Sequential(\n            nn.Linear(dim, 50),\n            nn.Tanh(),\n            nn.Linear(50, dim),\n        )\n        self.nfe = 0\n\n        for m in self.net.modules():\n            if isinstance(m, nn.Linear):\n                nn.init.normal_(m.weight, mean=0, std=0.1)\n                nn.init.constant_(m.bias, val=0)\n\n    def forward(self, t, x):\n        self.nfe += 1\n        return self.net(x)\n'

In [0]:
#test
a = torch.ones(3, 2)
b = torch.zeros(3, 3)
torch.cat([a, b], dim=1)

tensor([[1., 1., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 0., 0., 0.]])

In [0]:
#T_series = torch.linspace(0., 25., 100)

In [0]:
class ODEBlock(nn.Module):

    def __init__(self, odefunc):
        super(ODEBlock, self).__init__()
        self.odefunc = odefunc
        self.integration_time = torch.tensor([0, 1]).float()

    def forward(self, x):
        self.integration_time = self.integration_time.type_as(x)
        #batch_t = T_series[:x.shape[0]]
        out = odeint(self.odefunc, x, self.integration_time , rtol=Rtol, atol=Atol, method = 'adams')
        return out[1]

    @property
    def nfe(self):
        return self.odefunc.nfe

    @nfe.setter
    def nfe(self, value):
        self.odefunc.nfe = value

In [0]:
class Flatten(nn.Module):

    def __init__(self):
        super(Flatten, self).__init__()

    def forward(self, x):
        shape = torch.prod(torch.tensor(x.shape[1:])).item()
        ret = x.view(-1, shape)
        return ret


In [0]:
class RunningAverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self, momentum=0.99):
        self.momentum = momentum
        self.reset()

    def reset(self):
        self.val = None
        self.avg = 0

    def update(self, val):
        if self.val is None:
            self.avg = val
        else:
            self.avg = self.avg * self.momentum + val * (1 - self.momentum)
        self.val = val


In [0]:
def inf_generator(iterable):
    """Allows training with DataLoaders in a single infinite loop:
        for i, (x, y) in enumerate(inf_generator(train_loader)):
    """
    iterator = iterable.__iter__()
    while True:
        try:
            yield iterator.__next__()
        except StopIteration:
            iterator = iterable.__iter__()


def learning_rate_with_decay(batch_size, batch_denom, batches_per_epoch, boundary_epochs, decay_rates):
    initial_learning_rate = Lr * batch_size / batch_denom

    boundaries = [int(batches_per_epoch * epoch) for epoch in boundary_epochs]
    vals = [initial_learning_rate * decay for decay in decay_rates]

    def learning_rate_fn(itr):
        lt = [itr < b for b in boundaries] + [True]
        i = np.argmax(lt)
        return vals[i]

    return learning_rate_fn

In [0]:
def accuracy(model, dataset):
    x,y = data_loader(dataset=dataset, batch_turn=0, size=len(dataset))
    x = torch.FloatTensor(x)
    x = x.to(Device)
    y = y.numpy()
    result = model(x).cpu().detach().numpy()
    total_correct =  np.sum(np.square(result - y))
    return total_correct / len(dataset)

In [0]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [0]:
train_dataset, test_dataset, train_eval_dataset = generate_data(Batch_size)

In [0]:
input_size = 9 * (Embed_size)
feature_layers = [ODEBlock(ODEfunc(9))]
fc_layers = [Flatten(), nn.Linear(input_size, 2)]
model = nn.Sequential(*feature_layers, *fc_layers).to(Device)

criterion = nn.MSELoss().to(Device)

batches_per_epoch = len(train_dataset) // Batch_size

lr_fn = learning_rate_with_decay(
        Batch_size, batch_denom=128, batches_per_epoch=batches_per_epoch, boundary_epochs=[60, 100, 140],
        decay_rates=[1, 0.1, 0.01, 0.001]
)

optimizer = torch.optim.SGD(model.parameters(), lr=Lr, momentum=0.9)

best_acc = 0
batch_time_meter = RunningAverageMeter()
f_nfe_meter = RunningAverageMeter()
b_nfe_meter = RunningAverageMeter()
end = time.time()

# Train

In [0]:
for itr in range(NEpochs):
    for batch_turn in range(batches_per_epoch):
        x,y = data_loader(dataset=train_dataset, batch_turn=batch_turn, size=Batch_size)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr_fn(itr)
        optimizer.zero_grad()
        x = x.to(Device)
        y = y.to(Device)
        logits = model(x)
        result = logits.cpu().detach().numpy()
        loss = criterion(logits, y)

        if Is_odenet:
            nfe_forward = feature_layers[0].nfe
            feature_layers[0].nfe = 0

        loss.backward()
        optimizer.step()

        if Is_odenet:
            nfe_backward = feature_layers[0].nfe
            feature_layers[0].nfe = 0

        batch_time_meter.update(time.time() - end)
        if Is_odenet:
            f_nfe_meter.update(nfe_forward)
            b_nfe_meter.update(nfe_backward)
        end = time.time()

    random.shuffle(train_dataset)
    with torch.no_grad():
        train_acc = accuracy(model, train_eval_dataset)
        val_acc = accuracy(model, test_dataset)
        '''if val_acc > best_acc:
                torch.save({'state_dict': model.state_dict(), 'args': args}, os.path.join(args.save, 'model.pth'))
                best_acc = val_acc
            '''
        print(
                "Epoch {:04d} | Time {:.3f} ({:.3f}) | NFE-F {:.1f} | NFE-B {:.1f} | Train Acc {:.4f} | Test Acc {:.4f}".format( itr // batches_per_epoch, batch_time_meter.val, batch_time_meter.avg, f_nfe_meter.avg, b_nfe_meter.avg, train_acc, val_acc )
            )

# Predict input

adjoint.py의 81번째 코드 참고!

In [0]:
# predict x by y